<a href="https://colab.research.google.com/github/louispaulet/randomForestClassifierVisualizationProject/blob/main/TREE_app_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JupyterDash
The `jupyter-dash` package makes it easy to develop Plotly Dash apps from the Jupyter Notebook and JupyterLab.

Just replace the standard `dash.Dash` class with the `jupyter_dash.JupyterDash` subclass.

In [1]:
!pip install jupyter-dash

In [2]:
from jupyter_dash import JupyterDash

In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

In [4]:
import dash_table
from dash.dependencies import Input, Output, State


When running in JupyterHub or Binder, call the `infer_jupyter_config` function to detect the proxy configuration.

In [5]:
JupyterDash.infer_jupyter_proxy_config()

Load and preprocess data

In [6]:
df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
available_indicators = df['Indicator Name'].unique()

Construct the app and callbacks

In [7]:
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# df = pd.DataFrame()

# # Create server variable with Flask server object for use with gunicorn
# server = app.server

# app.layout = html.Div([
                       
#     html.H1(children='T.R.E.E : T.R.E.E Representation and Experimentation Engine'),

#     html.Div([
#       html.Div(dcc.Input(id='input-on-submit', type='text')),
#       html.Button('Submit', id='submit-val', n_clicks=0),
#       html.Div(id='container-button-basic',
#               children='Enter a value and press submit')
#     ]),
    
#     #display dataframe

#     # html.Div([
#     #       dash_table.DataTable(
#     #         id='table',
#     #         columns=[{"name": i, "id": i} for i in df.columns],
#     #         data=df.to_dict('records'),
#     #         )
#     # ]),   
# html.Div([
#         dash_table.DataTable(id='datatable-upload-container')
#         ]), 
# ])

# @app.callback(
#     # dash.dependencies.Output('container-button-basic', 'children'),
#     dash.dependencies.Output('datatable-upload-container', 'columns', 'data'),
#     [dash.dependencies.Input('submit-val', 'n_clicks')],
#     [dash.dependencies.State('input-on-submit', 'value')])
# def update_output(n_clicks, value):

#   if contents is None:
#           return [{}], []
#   df = pd.read_csv(value)

#   return df.to_dict('records'), [{"name": i, "id": i} for i in df.columns]

Serve the app using `run_server`.  Unlike the standard `Dash.run_server` method, the `JupyterDash.run_server` method doesn't block execution of the notebook. It serves the app in a background thread, making it possible to run other notebook calculations while the app is running.

This makes it possible to iterativly update the app without rerunning the potentially expensive data processing steps.

By default, `run_server` displays a URL that you can click on to open the app in a browser tab. The `mode` argument to `run_server` can be used to change this behavior.  Setting `mode="inline"` will display the app directly in the notebook output cell.

In [33]:
import base64
import io
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
html.Div([                       
    html.H1('T.R.E.E : T.R.E.E Representation and Experimentation Engine'),
]),
html.Div([                       

    dcc.Upload(
        id='datatable-upload',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%', 'height': '60px', 'lineHeight': '60px',
            'borderWidth': '1px', 'borderStyle': 'dashed',
            'borderRadius': '5px', 'textAlign': 'center', 'margin': '10px'
        },
    ),
    ]),
html.Div([                       
    html.H1(title='AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', id='title_df'),
    
    dash_table.DataTable(id='datatable-upload-container'),
]),
html.Div([  
    dcc.Graph(id='datatable-upload-graph')
    ]),
])


def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    if 'csv' in filename:
        # Assume that the user uploaded a CSV file
        return pd.read_csv(
            io.StringIO(decoded.decode('utf-8')))
    elif 'xls' in filename:
        # Assume that the user uploaded an excel file
        return pd.read_excel(io.BytesIO(decoded))

@app.callback(Output('title_df', 'title'),        
Input('datatable-upload', 'contents'))
def display_title_df():
  return '5 first rows of the dataset : '

@app.callback(Output('datatable-upload-container', 'data'),
              Output('datatable-upload-container', 'columns'),
              Output('datatable-upload-container', 'hiddenDf'),
              Input('datatable-upload', 'contents'),
              State('datatable-upload', 'filename'))
def update_output(contents, filename):
    if contents is None:
        return [{}], []
    df = parse_contents(contents, filename)
    display_df = df.head()
    return display_df.to_dict('records'), [{"name": i, "id": i} for i in df.columns], df.to_dict('records')


@app.callback(Output('datatable-upload-graph', 'figure'),
              Input('datatable-upload-container', 'hiddenDf'))
def display_graph(rows):
    df = pd.DataFrame(rows)

    if (df.empty or len(df.columns) < 1):
        return {
            'data': [{
                'x': [],
                'y': [],
                'type': 'bar'
            }]
        }
    return {
        'data': [{
            'x': df[df.columns[0]],
            'y': df[df.columns[1]],
            'type': 'bar'
        }]
    }


if __name__ == '__main__':
    app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>

In [34]:
app.run_server(mode="inline")


<IPython.core.display.Javascript object>